In [1]:
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests, datetime
pd.set_option('display.max_columns', None)

Definitions: https://www.fairfaxcounty.gov/maps/sites/maps/files/assets/documents/dta_assessment_tables.pdf

Tax info: https://www.fairfaxcounty.gov/taxes/real-estate/tax-rates

Dwelling data: https://data-fairfaxcountygis.opendata.arcgis.com/datasets/Fairfaxcountygis::tax-administrations-real-estate-dwelling-data/

Land data: https://data-fairfaxcountygis.opendata.arcgis.com/datasets/Fairfaxcountygis::tax-administrations-real-estate-land-data/explore

Assessed Values: https://data-fairfaxcountygis.opendata.arcgis.com/datasets/Fairfaxcountygis::tax-administrations-real-estate-assessed-values/

Legal data: https://data-fairfaxcountygis.opendata.arcgis.com/maps/tax-administrations-real-estate-legal-data/about

Owner data: https://data-fairfaxcountygis.opendata.arcgis.com/maps/tax-administrations-real-estate-property-owner-addresses

Sales data: https://data-fairfaxcountygis.opendata.arcgis.com/datasets/Fairfaxcountygis::tax-administrations-real-estate-sales-data/explore


In [2]:
# import data from pre-fetched csvs in subfolder .\fairfax
sales = pd.read_csv('.\\fairfax\\Tax_Administration_s_Real_Estate_-_Sales_Data.csv', parse_dates=['SALEDT']).drop(
    ['Unnamed: 0', 'OBJECTID', 'BOOK', 'PAGE', 'TAXYR'], axis=1)
land = pd.read_csv('.\\fairfax\\Tax_Administration_s_Real_Estate_-_Land_Data 112922.csv').drop(
    ['OBJECTID', 'TAXYR', 'LLINE', 'ACRES'], axis=1)
dwellings = pd.read_csv('.\\fairfax\\Tax_Administration_s_Real_Estate_-_Dwelling_Data 112922.csv').drop(
    ['OBJECTID', 'CreationDate', 'Creator', 'EditDate', 'Editor'], axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# create a subset of sales from recent history for more accurate price modeling
sales_2022 = sales[(sales['SALEDT'] <= '2022-12-31')
                    & (sales['SALEDT'] >= '2022-01-01')
                    & (sales['PRICE'] > 1000)
                    & (sales['SALEVAL_DESC'] == 'Valid and verified sale')
                  ]


sales_2022 = sales_2022[sales_2022['PRICE'] < np.percentile(sales_2022['PRICE'], q=99)] # drop very high outliers


In [4]:
# grab zip codes for parcels (some will be lost due to lack of data); then grab ACS census estimates for median income of those zip codes
zips = pd.read_csv('.\\fairfax\\Tax_Administration_s_Real_Estate_-_Legal_Data 112922.csv', usecols = ['PARID','ZIP1']).dropna()

req_url = 'https://api.census.gov/data/2021/acs/acs5?get=B19013_001E&for=zip%20code%20tabulation%20area:' + ','.join([str(round(x)) for x in zips['ZIP1'].unique()])
r = requests.get(req_url).json()[:]

incomes = pd.DataFrame(r).iloc[1:].rename(columns={0:'Median_income', 1:'ZIP1'}).astype('float')
incomes.loc[incomes['Median_income'] < 0, 'Median_income'] = incomes[incomes['Median_income'] > 0]['Median_income'].median()
zips = pd.merge(zips,incomes)
zips.drop(['ZIP1'], axis=1, inplace=True)

In [5]:
# merging land and dwelling data
dwellings = pd.merge(dwellings, land)
# merging land and dwelling data with zip code median income
dwellings = pd.merge(dwellings, zips)

In [6]:
# renaming ambiguously named columns
dwellings.rename(
    columns={'USER10':'Basement bedrooms/dens count',
            'USER6':'Model Name',
            'USER13_DESC':'Roof type',
            'USER7_DESC':'Dormer type',
            'USER9_DESC':'Basement type',}, inplace=True)
dwellings.dropna(subset=['GRADE_DESC'], inplace=True)

# cleaning and filling empty data
dwellings['RECROMAREA'].fillna(0, inplace=True)
dwellings['YRREMOD'].fillna(0, inplace=True)
dwellings['WBFP_PF'].fillna(0, inplace=True)
dwellings['BSMTCAR'].fillna(0, inplace=True)
dwellings['FIXHALF'].fillna(0, inplace=True)
dwellings['FIXBATH'].fillna(0, inplace=True)
dwellings['Basement bedrooms/dens count'].fillna(0, inplace=True)

dwellings = dwellings[~dwellings['RMBED'].isna()]

# dictionaries to map grade and condition descriptions to ordinal numeric values
grades = {'Commercial Grade' : 1, 'Average' : 2, 'Average 10' : 3, 'Good' : 4, 'Good 10' : 5,
          'Good 20' : 6, 'Excellent' : 7, 'Excellent 10' : 8, 'Excellent 20' : 9, 'Excellent 30' : 10,
          'Luxury' : 11, 'Luxury 10' : 12, 'Luxury 20' : 13, 'Luxury 30' : 14, 'Mansion' : 15,
          'Mansion 10' : 16, 'Mansion 20' : 17, 'Mansion 30' : 18}
cdus = {'Poor':1, 'Fair':2, 'Average':3, 'Average Plus':4, 'Good':5}

# mapping grades and property descriptions to numeric values
dwellings.loc[:, 'GRADE_DESC'] = dwellings['GRADE_DESC'].replace(grades)
dwellings.loc[:, 'CDU_DESC'] = dwellings['CDU_DESC'].replace(cdus)

# one hot encoding for certain categorical columns
dwellings = pd.get_dummies(dwellings, columns=['CODE_DESC'])
dwellings = pd.get_dummies(dwellings, columns=['HEAT_DESC'])
dwellings = pd.get_dummies(dwellings, columns=['BSMT_DESC'])

dwellings.drop(['EFFYR', 'Model Name', 'STYLE_DESC',
              'EXTWALL_DESC', 'Roof type', 'Dormer type', 'Basement type'], axis=1, inplace=True)
# print(dwellings.dtypes)

In [7]:
# drop low-importance features (determined by previous random forest estimates)
low_importance_features = ['BSMT_DESC_1/2 Bsmt',
 'BSMT_DESC_Sub & Full',
 'CODE_DESC_RESIDENTIAL STACKED TOWNHOUSE',
 'HEAT_DESC_Central',
 'UNITS',
 'HEAT_DESC_Central A/C',
 'CODE_DESC_DUPLEX',
 'HEAT_DESC_Non Central',
 'CODE_DESC_HOMESITE',
 'CODE_DESC_SINGLE DWELLING RESIDUAL',
 'CODE_DESC_OPEN SPACE',
 'CODE_DESC_NON-TAXABLE',
 'CODE_DESC_UNUSABLE',
 'HEAT_DESC_None',
 'CODE_DESC_FORESTRY',
 'CODE_DESC_AGRICULTURE',
 'CODE_DESC_MULTIFAMILY/ELDERLY']
dwellings.drop(low_importance_features, axis=1, inplace=True)


In [8]:
# creating dataframe for 
rf_data = pd.merge(dwellings, sales_2022).drop(['SALEVAL_DESC', 'SALEDT'], axis=1).dropna()
print(rf_data.shape)
rf_data.drop(['PARID'], axis=1, inplace=True)


(13878, 24)


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import RandomizedSearchCV

import scipy.stats as stats

y = np.log(rf_data.loc[:, 'PRICE'])
X = rf_data.loc[:, rf_data.columns != 'PRICE']

# split the data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
display(X_test.head())
display(y_test.head())

,YRBLT,YRREMOD,RMBED,FIXBATH,FIXHALF,RECROMAREA,WBFP_PF,BSMTCAR,Basement bedrooms/dens count,GRADE_DESC,SFLA,CDU_DESC,SF,Median_income,CODE_DESC_RESIDENTIAL,CODE_DESC_SINGLE DWELLING,CODE_DESC_TWNHSE END,CODE_DESC_TWNHSE INT,BSMT_DESC_1/3 Bsmt,BSMT_DESC_2/3 Bsmt,BSMT_DESC_Full,BSMT_DESC_None
4281,1998.0,0.0,3.0,3.0,1.0,252.0,0.0,2.0,0.0,3,1814.0,3.0,1760.0,140928.0,0,0,0,1,0,0,1,0
2856,1994.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,4,966.0,3.0,0.0,115486.0,1,0,0,0,0,0,0,1
2183,1972.0,2011.0,4.0,3.0,0.0,338.0,1.0,2.0,1.0,2,1422.0,5.0,8400.0,150701.0,0,1,0,0,0,0,1,0
13027,2001.0,0.0,5.0,3.0,1.0,785.0,1.0,0.0,1.0,6,3524.0,3.0,10500.0,240398.0,0,1,0,0,0,0,1,0
2627,2022.0,0.0,5.0,5.0,1.0,810.0,1.0,0.0,2.0,5,4068.0,3.0,8925.0,123525.0,0,1,0,0,0,0,1,0


4281     13.384728
2856     12.721886
2183     13.567049
13027    14.331921
2627     13.997217
Name: PRICE, dtype: float64

In [ ]:
# create dummy distributions for the random search algorithm
distributions = {
    'max_features': stats.norm(0.5, 0.010**0.5),
    'n_estimators': stats.randint(1, 1000),
}

# random forest regression cross validation/parameter optimization
random_search = RandomizedSearchCV(RandomForestRegressor(),
                     distributions, n_iter=10,
                          scoring = 'neg_mean_squared_error',
                          n_jobs = -1,
                          )
# Fitting the search
random_search.fit(X_train, y_train)

# Printing the results
print(random_search.best_params_)

In [27]:
rf = RandomForestRegressor(n_estimators=200)
rf.fit(X_train, y_train)

fi = pd.DataFrame({
        'feature': X.iloc[:,:].columns,
        'importance': rf.feature_importances_}).sort_values('importance', ascending=False).reset_index(drop=True).round(4)

fcst = rf.predict(X_test.iloc[:,:])

display(fi)


target = dwellings[dwellings['PARID'] == '0871 11  0009']

target.drop([x for x in target.columns if x not in X.columns], axis=1, inplace=True)

display(target)
print('Standard Dev of prices:{:.3f}\nR2: {:.3f}\nRMSE: {:.3f}\nMAE: {:.3f}\nMAPE: {:.1f}%\n\nRandom Forest Estimate: ${:,.0f}'.format(
    np.std(np.exp(y)),
    r2_score(list(y_test), list(fcst)),
    mean_squared_error(list(np.exp(y_test)), np.exp(list(fcst)))**0.5,
    mean_absolute_error(np.exp(list(y_test)), np.exp(list(fcst))),
    mean_absolute_percentage_error(np.exp(list(y_test)), np.exp(list(fcst)))* 100,
    np.exp(rf.predict(target)[0])
     ))


,feature,importance
0,SF,0.4754
1,SFLA,0.3310
2,GRADE_DESC,0.0926
3,FIXBATH,0.0318
4,Median_income,0.0232
5,YRBLT,0.0154
6,CODE_DESC_SINGLE DWELLING,0.0069
7,RECROMAREA,0.0061
8,YRREMOD,0.0027
9,BSMT_DESC_None,0.0026


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,YRBLT,YRREMOD,RMBED,FIXBATH,FIXHALF,RECROMAREA,WBFP_PF,BSMTCAR,Basement bedrooms/dens count,GRADE_DESC,SFLA,CDU_DESC,SF,Median_income,CODE_DESC_RESIDENTIAL,CODE_DESC_SINGLE DWELLING,CODE_DESC_TWNHSE END,CODE_DESC_TWNHSE INT,BSMT_DESC_1/3 Bsmt,BSMT_DESC_2/3 Bsmt,BSMT_DESC_Full,BSMT_DESC_None
92762,1992.0,0.0,4.0,4.0,1.0,1338.0,1.0,0.0,1.0,6,4368.0,3.0,220370.0,221899.0,0,1,0,0,0,0,1,0


Standard Dev of prices:436794.436
R2: 0.962
RMSE: 107213.660
MAE: 60181.132
MAPE: 7.3%

Random Forest Estimate: $1,304,843
